# Trabajo #1: Análisis de Face Tracking y Visión por Computadora
**Proyecto de Métodos Numéricos**

### 1. ¿Qué es MindAR?
MindAR es una biblioteca de software de código abierto y ligera diseñada para desarrollar experiencias de Realidad Aumentada (AR) en la web. Permite el reconocimiento de imágenes y seguimiento facial directamente en el navegador utilizando tecnologías estándar como WebGL y WebAssembly, eliminando la necesidad de instalar aplicaciones externas.

### 2. ¿Qué es OpenCV?
OpenCV (Open Source Computer Vision Library) es la biblioteca de visión artificial de código abierto más utilizada a nivel mundial. Provee una infraestructura común para aplicaciones de visión por computadora y contiene más de 2500 algoritmos optimizados para tareas como detección de rostros, identificación de objetos, clasificación de acciones en video, rastreo de movimientos y procesamiento de imágenes (filtros, bordes, transformaciones).

### 3. ¿De manera interna MindAR usa OpenCV?
**No.** Aunque ambas herramientas procesan imágenes, MindAR no depende de OpenCV.
* **MindAR** está construida sobre TensorFlow.js y utiliza modelos de aprendizaje profundo (Deep Learning) propietarios y ligeros para realizar la detección y seguimiento de características faciales o imágenes planas.
* **OpenCV** se basa en algoritmos clásicos de procesamiento de matrices de píxeles, mientras que MindAR se basa en inferencia de redes neuronales.

### 4. ¿Se puede utilizar OpenCV en JavaScript?
**Sí.** Existe una versión oficial llamada OpenCV.js. Mediante la tecnología WebAssembly (Wasm), el código original de C++ de OpenCV es compilado para que pueda ser ejecutado directamente por el navegador web (lado del cliente) con un rendimiento cercano al nativo, permitiendo realizar procesamiento de imágenes complejo en tiempo real dentro de páginas web.

### 5. ¿Para qué sirve el algoritmo de Canny Edge Detection?
El algoritmo de Canny es una técnica de procesamiento de imágenes utilizada para detectar bordes de manera robusta. Es considerado el algoritmo estándar óptimo para esta tarea porque cumple tres criterios clave:
1.  **Detección:** Baja tasa de error (encuentra todos los bordes reales).
2.  **Localización:** Los puntos detectados deben estar lo más cerca posible del borde real.
3.  **Respuesta única:** Debe marcar una sola línea por cada borde (evita bordes gruesos o múltiples respuestas al mismo contorno).

### 6. Ejemplo del algoritmo de Canny Edge Detection en JavaScript

# Análisis Matemático: Implementación "Desde Cero" de Canny Edge

En esta versión final, nuestro equipo decidió no depender de librerías externas para el procesamiento central. Hemos programado el algoritmo de Canny manualmente, manipulando los arrays de píxeles (`Uint8ClampedArray`) directamente. Esto nos permitió aplicar las ecuaciones matemáticas de visión artificial de forma explícita en cada etapa.

### 1. Pre-procesamiento: Luminancia (Espacio Vectorial)
La primera operación matemática es reducir la dimensionalidad de la imagen de $\mathbb{R}^3$ (RGB) a $\mathbb{R}^1$ (Grises). No usamos un promedio simple; aplicamos la ecuación de **Luminancia Percibida** que pondera los canales según la sensibilidad del ojo humano:

$$
I(x,y) = 0.299 \cdot R + 0.587 \cdot G + 0.114 \cdot B
$$

> **En nuestro código:** Iteramos sobre el buffer de datos crudos (`imageData.data`) aplicando esta combinación lineal punto a punto.

---

### 2. Suavizado Gaussiano (Convolución Discreta)
Para eliminar el ruido de alta frecuencia, aplicamos una operación de convolución con un Kernel Gaussiano de $5 \times 5$.

Matemáticamente, el valor de cada píxel suavizado $S(x,y)$ es la suma ponderada de sus vecinos, definida por la matriz de convolución $K$:

$$
S(x,y) = \frac{1}{159} \sum_{i=-2}^{2} \sum_{j=-2}^{2} I(x+i, y+j) \cdot K(i,j)
$$

Donde $159$ es el factor de normalización (la suma de todos los elementos del kernel) para mantener el brillo original.

---

### 3. Cálculo de Gradientes (Diferenciación Numérica)
Implementamos manualmente los operadores de Sobel para calcular las derivadas parciales. Para cada píxel, calculamos:
* **Magnitud del Gradiente ($|G|$):** La hipotenusa del vector gradiente.
  $$|G| = \sqrt{G_x^2 + G_y^2}$$
* **Dirección del Gradiente ($\theta$):** El ángulo de orientación del borde.
  $$\theta = \arctan\left(\frac{G_y}{G_x}\right)$$

> **En nuestro código:** Usamos `Math.sqrt` y `Math.atan2` sobre los resultados de las máscaras de convolución vertical y horizontal.

---

### 4. Supresión de No-Máximos (Discretización de Ángulos)
Para adelgazar los bordes a 1 píxel de ancho, realizamos una comparación direccional. Como los píxeles están en una rejilla cuadrada, discretizamos el ángulo continuo $\theta$ en 4 sectores principales:
* **Horizontal:** $0^\circ$
* **Vertical:** $90^\circ$
* **Diagonal Principal:** $45^\circ$
* **Diagonal Invertida:** $135^\circ$

La lógica matemática aplicada es:
$$
NMS(x,y) = \begin{cases} 
|G(x,y)| & \text{si } |G(x,y)| \ge |G(vecinos_{\theta})| \\
0 & \text{en caso contrario}
\end{cases}
$$
Esto elimina cualquier píxel que no sea el "pico" local de intensidad en la dirección del borde.

---

### 5. Histéresis y Conectividad (Lógica Topológica)
Finalmente, clasificamos los bordes usando dos umbrales escalares ($T_{high} = 90$ y $T_{low} = 30$):
1.  **Bordes Fuertes ($> T_{high}$):** Se aceptan incondicionalmente ($255$).
2.  **Bordes Débiles ($T_{low} < x < T_{high}$):** Se marcan temporalmente ($128$).
3.  **Ruido ($< T_{low}$):** Se descartan ($0$).

Para resolver los "Bordes Débiles", aplicamos un análisis de vecindad de 8-conexión. Un borde débil sobrevive **solo si** existe al menos un vecino fuerte en su vecindad inmediata ($3 \times 3$), garantizando la continuidad topológica de los contornos detectados.

In [17]:
%%html
<div style="text-align: center; background: #1a1a1a; padding: 20px; border-radius: 15px; color: white; font-family: sans-serif;">
    <h3>Método Canny Edge(Derivadas y Bordes)</h3>
    <video id="v_canny_pro" width="640" height="480" style="display:none" playsinline></video>
    <canvas id="c_canny_pro" width="640" height="480" style="background: #000; border: 2px solid #27ae60; border-radius: 10px;"></canvas>
    <div style="margin-top: 15px;">
        <button id="btn_on_canny" onclick="iniciarCannyPro()" style="padding: 10px 20px; background: #27ae60; color: white; border: none; border-radius: 5px; cursor: pointer; font-weight: bold;">ACTIVAR CANNY</button>
        <button id="btn_off_canny" onclick="detenerTodo()" style="padding: 10px 20px; background: #e74c3c; color: white; border: none; border-radius: 5px; cursor: pointer; margin-left: 10px; display: none;">APAGAR</button>
    </div>
    <p id="log_canny_pro" style="color: #f1c40f; font-size: 13px; margin-top: 10px;">Estado: Esperando cámara...</p>
</div>

<script>
// SISTEMA GLOBAL DE GESTIÓN DE CÁMARA
if (typeof window.cameraManager === 'undefined') {
    window.cameraManager = {
        currentStream: null,
        currentFilter: null,
        activeLoop: null
    };
}

function detenerTodo() {
    if (window.cameraManager.activeLoop) {
        window.cameraManager.activeLoop = false;
    }
    
    if (window.cameraManager.currentStream) {
        window.cameraManager.currentStream.getTracks().forEach(t => t.stop());
        window.cameraManager.currentStream = null;
    }
    
    // LIMPIAR TODOS LOS CANVAS
    ['c_canny_pro', 'c_out', 'c_sobel_pro'].forEach(canvasId => {
        const canvas = document.getElementById(canvasId);
        if (canvas) {
            const ctx = canvas.getContext('2d');
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            ctx.fillStyle = '#000';
            ctx.fillRect(0, 0, canvas.width, canvas.height);
        }
    });
    
    ['btn_off_canny', 'btn_off_sobel', 'b_stop'].forEach(id => {
        const btn = document.getElementById(id);
        if (btn) btn.style.display = 'none';
    });
    
    ['btn_on_canny', 'btn_on_sobel', 'b_start'].forEach(id => {
        const btn = document.getElementById(id);
        if (btn) btn.style.display = 'inline-block';
    });
    
    ['log_canny_pro', 'log_sobel_pro', 'debug_log'].forEach(id => {
        const log = document.getElementById(id);
        if (log) log.innerText = "Cámara liberada.";
    });
    
    window.cameraManager.currentFilter = null;
}

// Implementación REAL de Canny Edge Detection
function applyCannyEdgeDetection(imageData) {
    const width = imageData.width;
    const height = imageData.height;
    const data = imageData.data;
    
    // 1. Convertir a escala de grises
    const gray = new Uint8ClampedArray(width * height);
    for (let i = 0; i < data.length; i += 4) {
        const idx = i / 4;
        gray[idx] = 0.299 * data[i] + 0.587 * data[i + 1] + 0.114 * data[i + 2];
    }
    
    // 2. Suavizado Gaussiano 5x5
    const smoothed = new Uint8ClampedArray(width * height);
    const gaussianKernel = [
        2, 4, 5, 4, 2,
        4, 9, 12, 9, 4,
        5, 12, 15, 12, 5,
        4, 9, 12, 9, 4,
        2, 4, 5, 4, 2
    ];
    const kernelSum = 159;
    
    for (let y = 2; y < height - 2; y++) {
        for (let x = 2; x < width - 2; x++) {
            let sum = 0;
            for (let ky = -2; ky <= 2; ky++) {
                for (let kx = -2; kx <= 2; kx++) {
                    const idx = (y + ky) * width + (x + kx);
                    sum += gray[idx] * gaussianKernel[(ky + 2) * 5 + (kx + 2)];
                }
            }
            smoothed[y * width + x] = sum / kernelSum;
        }
    }
    
    // 3. Calcular gradientes con Sobel
    const gradX = new Float32Array(width * height);
    const gradY = new Float32Array(width * height);
    const magnitude = new Float32Array(width * height);
    const direction = new Float32Array(width * height);
    
    for (let y = 1; y < height - 1; y++) {
        for (let x = 1; x < width - 1; x++) {
            const idx = y * width + x;
            
            // Sobel X
            const gx = (
                -smoothed[(y-1)*width + (x-1)] + smoothed[(y-1)*width + (x+1)] +
                -2*smoothed[y*width + (x-1)] + 2*smoothed[y*width + (x+1)] +
                -smoothed[(y+1)*width + (x-1)] + smoothed[(y+1)*width + (x+1)]
            );
            
            // Sobel Y
            const gy = (
                -smoothed[(y-1)*width + (x-1)] - 2*smoothed[(y-1)*width + x] - smoothed[(y-1)*width + (x+1)] +
                smoothed[(y+1)*width + (x-1)] + 2*smoothed[(y+1)*width + x] + smoothed[(y+1)*width + (x+1)]
            );
            
            gradX[idx] = gx;
            gradY[idx] = gy;
            magnitude[idx] = Math.sqrt(gx * gx + gy * gy);
            direction[idx] = Math.atan2(gy, gx);
        }
    }
    
    // 4. Supresión no-máxima (esto hace que Canny sea diferente de Sobel)
    const suppressed = new Float32Array(width * height);
    
    for (let y = 1; y < height - 1; y++) {
        for (let x = 1; x < width - 1; x++) {
            const idx = y * width + x;
            const angle = direction[idx] * 180 / Math.PI;
            const mag = magnitude[idx];
            
            let n1 = 0, n2 = 0;
            
            // Determinar vecinos según la dirección del gradiente
            if ((angle >= -22.5 && angle < 22.5) || (angle >= 157.5 || angle < -157.5)) {
                // Horizontal
                n1 = magnitude[idx - 1];
                n2 = magnitude[idx + 1];
            } else if ((angle >= 22.5 && angle < 67.5) || (angle >= -157.5 && angle < -112.5)) {
                // Diagonal /
                n1 = magnitude[(y-1)*width + (x+1)];
                n2 = magnitude[(y+1)*width + (x-1)];
            } else if ((angle >= 67.5 && angle < 112.5) || (angle >= -112.5 && angle < -67.5)) {
                // Vertical
                n1 = magnitude[(y-1)*width + x];
                n2 = magnitude[(y+1)*width + x];
            } else {
                // Diagonal \
                n1 = magnitude[(y-1)*width + (x-1)];
                n2 = magnitude[(y+1)*width + (x+1)];
            }
            
            // Suprimir si no es máximo local
            if (mag >= n1 && mag >= n2) {
                suppressed[idx] = mag;
            } else {
                suppressed[idx] = 0;
            }
        }
    }
    
    // 5. Umbralización con histéresis (doble umbral + seguimiento de bordes)
    const lowThreshold = 30;
    const highThreshold = 90;
    const edges = new Uint8ClampedArray(width * height);
    
    // Marcar píxeles fuertes
    for (let i = 0; i < suppressed.length; i++) {
        if (suppressed[i] >= highThreshold) {
            edges[i] = 255; // Borde fuerte
        } else if (suppressed[i] >= lowThreshold) {
            edges[i] = 128; // Borde débil (candidato)
        } else {
            edges[i] = 0;
        }
    }
    
    // Seguimiento de bordes (conectar bordes débiles a fuertes)
    for (let y = 1; y < height - 1; y++) {
        for (let x = 1; x < width - 1; x++) {
            const idx = y * width + x;
            
            if (edges[idx] === 128) { // Borde débil
                // Verificar si está conectado a un borde fuerte
                let connected = false;
                for (let dy = -1; dy <= 1; dy++) {
                    for (let dx = -1; dx <= 1; dx++) {
                        if (edges[(y+dy)*width + (x+dx)] === 255) {
                            connected = true;
                            break;
                        }
                    }
                    if (connected) break;
                }
                
                edges[idx] = connected ? 255 : 0;
            }
        }
    }
    
    // Convertir a ImageData
    const output = new ImageData(width, height);
    for (let i = 0; i < edges.length; i++) {
        output.data[i * 4] = edges[i];
        output.data[i * 4 + 1] = edges[i];
        output.data[i * 4 + 2] = edges[i];
        output.data[i * 4 + 3] = 255;
    }
    
    return output;
}

async function iniciarCannyPro() {
    const log = document.getElementById('log_canny_pro');
    
    detenerTodo();
    
    try {
        log.innerText = "Solicitando cámara...";
        
        const stream = await navigator.mediaDevices.getUserMedia({ 
            video: { width: 640, height: 480, facingMode: 'user' } 
        });
        
        window.cameraManager.currentStream = stream;
        window.cameraManager.currentFilter = 'canny';
        
        const v = document.getElementById('v_canny_pro');
        const canvas = document.getElementById('c_canny_pro');
        const ctx = canvas.getContext('2d');
        
        v.srcObject = stream;
        await v.play();
        
        await new Promise(resolve => setTimeout(resolve, 500));
        
        document.getElementById('btn_on_canny').style.display = 'none';
        document.getElementById('btn_off_canny').style.display = 'inline-block';
        log.innerText = "Detección de bordes Canny activa";
        
        window.cameraManager.activeLoop = true;
        let frameCount = 0;
        
        function processFrame() {
            if (!window.cameraManager.activeLoop || window.cameraManager.currentFilter !== 'canny') {
                return;
            }
            
            try {
                ctx.drawImage(v, 0, 0, canvas.width, canvas.height);
                const imageData = ctx.getImageData(0, 0, canvas.width, canvas.height);
                
                // Aplicar CANNY REAL
                const edges = applyCannyEdgeDetection(imageData);
                
                ctx.putImageData(edges, 0, 0);
                
                frameCount++;
                if (frameCount % 30 === 0) {
                    log.innerText = "Canny activo - Frames: " + frameCount;
                }
                
                requestAnimationFrame(processFrame);
                
            } catch (e) {
                console.error('Error:', e);
                log.innerText = "Error: " + e.message;
            }
        }
        
        processFrame();
        
    } catch (e) {
        log.innerText = "Error: " + e.message;
        console.error(e);
    }
}
</script>


### 7. ¿El algoritmo Canny Edge Detection utiliza derivadas?
[cite_start]**Sí.** En el contexto de una imagen, un borde se define matemáticamente como un cambio brusco en la intensidad de los píxeles[cite: 34]. [cite_start]El algoritmo de Canny busca los puntos donde la primera derivada de la función de intensidad de la imagen alcanza un máximo local (es decir, donde el gradiente es más pronunciado)[cite: 35].

### 8. ¿En qué forma el algoritmo Canny Edge utiliza las diferencias finitas en su cálculo?
[cite_start]Dado que una imagen digital es una matriz discreta de píxeles y no una función continua, no es posible calcular derivadas analíticas[cite: 38]. [cite_start]El algoritmo utiliza **Diferencias Finitas** para aproximar el gradiente[cite: 39]. [cite_start]Se aplican núcleos de convolución (como el operador Sobel) que realizan restas ponderadas entre píxeles vecinos (ej. $f(x+1)-f(x-1)$) para estimar la tasa de cambio (derivada) en las direcciones horizontal y vertical[cite: 39].

### 9. Algoritmos para detectar bordes
[cite_start]Existen diversos operadores basados en el cálculo del gradiente a través de diferencias finitas[cite: 41]:
* [cite_start]Operador Sobel [cite: 42]
* [cite_start]Operador Prewitt [cite: 43]
* [cite_start]Operador Roberts [cite: 44]
* [cite_start]Laplaciano de Gaussiana (LOG) [cite: 45]
* [cite_start]Algoritmo de Canny [cite: 46]

### 10. ¿Para qué sirve el algoritmo de Sobel?
[cite_start]El operador Sobel sirve para calcular una aproximación del gradiente de intensidad de una imagen[cite: 48]. [cite_start]Se utiliza para detectar bordes resaltando las regiones de alta frecuencia espacial[cite: 49]. [cite_start]Es computacionalmente eficiente y efectivo para detectar la orientación y magnitud de los bordes simples[cite: 50].

### 11 y 12. ¿Cómo utiliza Sobel las derivadas?
[cite_start]Calcula la **Primera Derivada discreta** usando máscaras de $3\times3$ para $G_{x}$ (horizontal) y $G_{y}$ (vertical)[cite: 52].
La magnitud total se obtiene con:
[cite_start]$$G=\sqrt{G_{x}^{2}+G_{y}^{2}}$$ [cite: 53]

### 13. Relación de Sobel con diferencias finitas
La relación es directa. [cite_start]El núcleo aplica una **Diferencia Central** combinada con un suavizado[cite: 55]. [cite_start]Por ejemplo, el kernel `[-1, 0, +1]` es la definición numérica de la primera diferencia finita[cite: 56].

### 14. ¿Se requiere escala de grises para Sobel?
[cite_start]**Sí.** Los algoritmos de detección de bordes operan sobre cambios de intensidad (luminosidad), no sobre información cromática[cite: 59]. [cite_start]Convertir la imagen a escala de grises simplifica la entrada de 3 canales (RGB) a 1 canal, reduciendo la complejidad computacional y eliminando el ruido que podrían introducir las variaciones de tono[cite: 60].

### 15. ¿MindAR es de fuente abierta?
[cite_start]**Sí.** MindAR se distribuye bajo la licencia MIT[cite: 62]. [cite_start]Esto significa que es software libre y de código abierto, permitiendo su uso, modificación y distribución tanto para proyectos personales como comerciales sin restricciones significativas[cite: 63].

### 16. ¿Qué es MediaPipe Face Mesh de Google?
[cite_start]MediaPipe Face Mesh es una solución de aprendizaje automático (Machine Learning) desarrollada por Google que permite la estimación geométrica de rostros en tiempo real[cite: 65]. [cite_start]Es capaz de detectar **468 puntos de referencia (landmarks)** en 3D sobre el rostro humano, funcionando eficientemente incluso en dispositivos móviles sin hardware dedicado[cite: 66].

### 17, 20, 21 y 22. Implementación Técnica: Face Mesh con Filtros y WebCam
[cite_start]A continuación se presenta el código fuente que integra los requerimientos[cite: 76]:
* [cite_start]**Punto 17:** Carga MediaPipe Face Mesh y dibuja los 468 puntos[cite: 78].
* [cite_start]**Punto 20:** Solicitud de acceso a webcam[cite: 79].
* [cite_start]**Punto 21:** Edición de máscara colocando un objeto[cite: 80].
* [cite_start]**Punto 22:** Filtro en un punto específico (nariz)[cite: 81].

# Análisis Matemático: Motor de Renderizado AR Vectorial

Para este módulo final, nuestro equipo desarrolló un sistema de **Realidad Aumentada (AR)** que no depende de la superposición de imágenes estáticas (sprites). En su lugar, hemos programado un motor de renderizado geométrico que construye los objetos píxel a píxel en tiempo real, utilizando ecuaciones paramétricas y transformaciones lineales.

A continuación, detallamos la lógica matemática que gobierna nuestro código:

### 1. Transformación de Espacios (Mapeo Lineal)
El modelo de Inteligencia Artificial (MediaPipe Face Mesh) nos devuelve los puntos clave (landmarks) en un **Espacio Normalizado** $\mathbb{R}^2_{[0,1]}$. Para poder dibujar en la pantalla, aplicamos una **Transformación Afín de Escala** para mapear estos valores al espacio discreto del Canvas ($640 \times 480$).

Para cualquier punto $P_n(x, y)$ detectado por la red neuronal, su posición en pantalla $P_s$ se calcula como:

$$
P_s = \begin{bmatrix} W_{canvas} & 0 \\ 0 & H_{canvas} \end{bmatrix} \cdot P_n
$$

> **En nuestro código:** Esto se evidencia en instrucciones como `nose.x * can_el.width`.

---

### 2. Escalamiento Dinámico (Simulación de Profundidad Z)
Uno de los desafíos matemáticos fue lograr que los objetos (lentes, sombreros) cambiaran de tamaño coherentemente al acercarnos o alejarnos de la cámara. Sin un sensor de profundidad (LiDAR), utilizamos una **Métrica Relativa Euclideana**.

Seleccionamos dos puntos anatómicamente rígidos: los pómulos izquierdo y derecho (Landmarks **#454** y **#234**). Calculamos la magnitud del vector que los une para derivar un factor de escala $k$:

$$
k = |x_{454} - x_{234}| \cdot W_{canvas}
$$

Este escalar $k$ se propaga a todas las funciones de dibujo. Por ejemplo, el radio de la nariz se define como $r = 0.15 \cdot k$. Esto garantiza que la geometría mantenga su **Proporcionalidad Homotética** sin importar la distancia del sujeto.

---

### 3. Construcción de Primitivas Geométricas
En lugar de cargar texturas, utilizamos **Geometría Analítica** para dibujar los filtros:

#### A. Simulación Volumétrica (Nariz 3D)
Para la "Nariz Roja", no dibujamos un círculo plano. Para simular una esfera 3D, implementamos una función de **Interpolación Radial de Color** (Gradiente).
Matemáticamente, definimos una función de intensidad $I(r)$ que decae desde un punto focal desplazado $(x - \Delta, y - \Delta)$ hacia el borde. Esto simula la **Reflexión Especular** (brillo) de una fuente de luz, engañando al ojo para percibir volumen esférico.

#### B. Polígonos y Vectores de Traslación (Corona y Sombrero)
Para objetos complejos como la corona, definimos polígonos irregulares mediante una secuencia de vértices $V = \{v_1, v_2, ..., v_n\}$.
El reto es ubicar estos objetos *sobre* la cabeza, no *en* ella. Para ello, aplicamos un **Vector de Traslación Vertical** relativo al punto de anclaje (la frente, Landmark #10):

$$
P_{objeto} = P_{frente} + \vec{v}_{offset}
$$

Donde $\vec{v}_{offset} = (0, -0.6 \cdot k)$. El signo negativo indica un desplazamiento hacia arriba en el sistema de coordenadas de la pantalla (donde Y crece hacia abajo).

---

### 4. Topología de Malla (Grafos)
Finalmente, la "Malla Verde" que visualizamos mediante `drawConnectors` representa la **Matriz de Adyacencia** del grafo facial.
El rostro se modela como un grafo $G=(V,E)$, donde $V$ son los 468 vértices detectados y $E$ son las aristas que definen la triangulación de la superficie, permitiéndonos visualizar la estructura topológica que la IA "entiende" del rostro humano.



In [1]:
%%html
<div id="ar_final_box" style="text-align: center; background: #1a1a1a; padding: 20px; border-radius: 15px; color: white; font-family: sans-serif;">
    <h2 style="color: #3498db;">PROYECTO FINAL: Filtros AR 3D</h2>
    
    <div style="margin-bottom: 10px; display: flex; justify-content: center; gap: 5px; flex-wrap: wrap;">
        <button onclick="setF(0)">Quitar</button>
        <button onclick="setF(1)">Nariz Roja</button>
        <button onclick="setF(2)">Sombrero</button>
        <button onclick="setF(3)">Lentes</button>
        <button onclick="setF(4)">Corona</button>
    </div>

    <div style="margin-bottom: 15px;">
        <button id="m_btn" onclick="togM()" style="padding: 8px 15px; background: #8e44ad; color: white; border: none; border-radius: 5px; cursor: pointer;">Ocultar Malla Verde</button>
    </div>

    <div style="position: relative; display: inline-block;">
        <video id="v_src" style="display:none" playsinline></video>
        <canvas id="c_out" width="640" height="480" style="background: #000; border: 2px solid #444; border-radius: 10px;"></canvas>
    </div>
    
    <div style="margin-top: 15px;">
        <button id="b_start" onclick="runAr()" style="padding: 15px 30px; background: #27ae60; color: white; border: none; border-radius: 10px; cursor: pointer; font-weight: bold;">ACTIVAR FILTROS AR</button>
        <button onclick="detenerTodo()" style="padding: 15px 30px; background: #e74c3c; color: white; border: none; border-radius: 10px; cursor: pointer; margin-left: 10px; display: none;" id="b_stop">APAGAR</button>
        <p id="debug_log" style="color: #f1c40f; font-size: 13px; margin-top: 10px; background: #000; padding: 5px;">Estado: Esperando clic...</p>
    </div>
</div>

<script src="https://cdn.jsdelivr.net/npm/@mediapipe/camera_utils/camera_utils.js" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/face_mesh.js" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/drawing_utils/drawing_utils.js" crossorigin="anonymous"></script>

<script>
var vid_el = document.getElementById('v_src');
var can_el = document.getElementById('c_out');
var ctx_el = can_el.getContext('2d');
var log_el = document.getElementById('debug_log');

var show_m = true;
var filter_id = 0;
var arCamera = null;

function setF(n) {
    filter_id = n;
    log_el.innerText = n > 0 ? "Filtro " + n + " activado ✓" : "Filtro removido";
}

function togM() {
    show_m = !show_m;
    document.getElementById('m_btn').innerText = show_m ? "Ocultar Malla Verde" : "Mostrar Malla Verde";
}

// FUNCIONES PARA DIBUJAR FILTROS
function drawClownNose(ctx, x, y, size) {
    ctx.save();
    ctx.shadowColor = 'rgba(0,0,0,0.4)';
    ctx.shadowBlur = 25;
    ctx.shadowOffsetX = 15;
    ctx.shadowOffsetY = 15;
    
    const gradient = ctx.createRadialGradient(x - size*0.2, y - size*0.2, 0, x, y, size*0.6);
    gradient.addColorStop(0, '#ff6b6b');
    gradient.addColorStop(0.5, '#ee4444');
    gradient.addColorStop(1, '#cc2222');
    
    ctx.fillStyle = gradient;
    ctx.beginPath();
    ctx.arc(x, y, size*0.25, 0, Math.PI * 2);
    ctx.fill();
    
    ctx.shadowColor = 'transparent';
    const highlightGrad = ctx.createRadialGradient(x - size*0.15, y - size*0.15, 0, x - size*0.15, y - size*0.15, size*0.25);
    highlightGrad.addColorStop(0, 'rgba(255,255,255,0.9)');
    highlightGrad.addColorStop(1, 'rgba(255,255,255,0)');
    ctx.fillStyle = highlightGrad;
    ctx.beginPath();
    ctx.arc(x - size*0.15, y - size*0.15, size*0.25, 0, Math.PI * 2);
    ctx.fill();
    
    ctx.fillStyle = 'rgba(255,255,255,0.6)';
    ctx.beginPath();
    ctx.arc(x + size*0.1, y + size*0.15, size*0.08, 0, Math.PI * 2);
    ctx.fill();
    ctx.restore();
}

function drawHat(ctx, x, y, size) {
    ctx.save();
    ctx.fillStyle = '#8B4513';
    ctx.strokeStyle = '#654321';
    ctx.lineWidth = 3;
    ctx.shadowColor = 'rgba(0,0,0,0.5)';
    ctx.shadowBlur = 15;
    
    ctx.beginPath();
    ctx.ellipse(x, y, size*0.8, size*0.25, 0, 0, Math.PI * 2);
    ctx.fill();
    ctx.stroke();
    
    ctx.fillStyle = '#A0522D';
    ctx.beginPath();
    ctx.moveTo(x - size*0.4, y);
    ctx.lineTo(x - size*0.35, y - size*0.8);
    ctx.lineTo(x + size*0.35, y - size*0.8);
    ctx.lineTo(x + size*0.4, y);
    ctx.closePath();
    ctx.fill();
    ctx.stroke();
    
    ctx.fillStyle = '#FFD700';
    ctx.fillRect(x - size*0.35, y - size*0.3, size*0.7, size*0.15);
    ctx.restore();
}

function drawGlasses(ctx, x, y, size) {
    ctx.save();
    ctx.strokeStyle = '#000000';
    ctx.lineWidth = 4;
    ctx.shadowColor = 'rgba(0,0,0,0.5)';
    ctx.shadowBlur = 10;
    
    ctx.fillStyle = 'rgba(0,0,0,0.3)';
    ctx.beginPath();
    ctx.arc(x - size*0.35, y, size*0.25, 0, Math.PI * 2);
    ctx.fill();
    ctx.stroke();
    
    ctx.beginPath();
    ctx.arc(x + size*0.35, y, size*0.25, 0, Math.PI * 2);
    ctx.fill();
    ctx.stroke();
    
    ctx.beginPath();
    ctx.moveTo(x - size*0.1, y);
    ctx.lineTo(x + size*0.1, y);
    ctx.stroke();
    
    ctx.fillStyle = 'rgba(255,255,255,0.6)';
    ctx.beginPath();
    ctx.arc(x - size*0.42, y - size*0.08, size*0.08, 0, Math.PI * 2);
    ctx.fill();
    ctx.beginPath();
    ctx.arc(x + size*0.28, y - size*0.08, size*0.08, 0, Math.PI * 2);
    ctx.fill();
    ctx.restore();
}

function drawCrown(ctx, x, y, size) {
    ctx.save();
    ctx.fillStyle = '#FFD700';
    ctx.strokeStyle = '#FFA500';
    ctx.lineWidth = 3;
    ctx.shadowColor = 'rgba(0,0,0,0.5)';
    ctx.shadowBlur = 15;
    
    ctx.beginPath();
    ctx.moveTo(x - size*0.5, y);
    ctx.lineTo(x - size*0.4, y - size*0.4);
    ctx.lineTo(x - size*0.25, y - size*0.2);
    ctx.lineTo(x, y - size*0.5);
    ctx.lineTo(x + size*0.25, y - size*0.2);
    ctx.lineTo(x + size*0.4, y - size*0.4);
    ctx.lineTo(x + size*0.5, y);
    ctx.closePath();
    ctx.fill();
    ctx.stroke();
    
    const jewels = [
        {x: x - size*0.4, y: y - size*0.4, c: '#ff0000'},
        {x: x - size*0.25, y: y - size*0.2, c: '#00ff00'},
        {x: x, y: y - size*0.5, c: '#ff0000'},
        {x: x + size*0.25, y: y - size*0.2, c: '#0000ff'},
        {x: x + size*0.4, y: y - size*0.4, c: '#ff00ff'}
    ];
    
    jewels.forEach(j => {
        ctx.fillStyle = j.c;
        ctx.beginPath();
        ctx.arc(j.x, j.y, size*0.05, 0, Math.PI * 2);
        ctx.fill();
    });
    ctx.restore();
}

async function runAr() {
    log_el.innerText = "Paso 1: Solicitando cámara...";
    
    // Detener otros filtros
    detenerTodo();
    
    try {
        const stream = await navigator.mediaDevices.getUserMedia({ video: { width: 640, height: 480 } });
        window.cameraManager.currentStream = stream;
        window.cameraManager.currentFilter = 'ar';
        
        vid_el.srcObject = stream;
        await vid_el.play();
        
        document.getElementById('b_start').style.display = 'none';
        document.getElementById('b_stop').style.display = 'inline-block';
        
        log_el.innerText = "Paso 2: Cámara activa. Cargando IA...";
        setTimeout(startIA, 500);
    } catch (e) {
        log_el.innerText = "ERROR: No se pudo abrir la cámara. ¿Diste permiso?";
        console.error(e);
    }
}

function startIA() {
    try {
        const mesh = new FaceMesh({
            locateFile: (f) => `https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/${f}`
        });
        
        mesh.setOptions({ 
            maxNumFaces: 1, 
            refineLandmarks: true, 
            minDetectionConfidence: 0.5,
            minTrackingConfidence: 0.5
        });
        
        mesh.onResults((res) => {
            if (window.cameraManager.currentFilter !== 'ar') return;
            
            ctx_el.save();
            ctx_el.clearRect(0, 0, can_el.width, can_el.height);
            ctx_el.drawImage(res.image, 0, 0, can_el.width, can_el.height);
            
            if (res.multiFaceLandmarks && res.multiFaceLandmarks.length > 0) {
                const face = res.multiFaceLandmarks[0];
                
                if (show_m) {
                    drawConnectors(ctx_el, face, FACEMESH_TESSELATION, {color: '#00FF0050', lineWidth: 1});
                }

                const faceW = Math.abs(face[454].x - face[234].x) * can_el.width;
                
                if (filter_id === 1) {
                    const nose = face[1];
                    drawClownNose(ctx_el, nose.x * can_el.width, nose.y * can_el.height, faceW * 0.15);
                } 
                else if (filter_id === 2) {
                    const top = face[10];
                    drawHat(ctx_el, top.x * can_el.width, (top.y * can_el.height) - faceW*0.6, faceW * 0.8);
                }
                else if (filter_id === 3) {
                    const eyes = face[168];
                    drawGlasses(ctx_el, eyes.x * can_el.width, eyes.y * can_el.height, faceW);
                }
                else if (filter_id === 4) {
                    const top = face[10];
                    drawCrown(ctx_el, top.x * can_el.width, (top.y * can_el.height) - faceW*0.5, faceW * 0.6);
                }
            }
            ctx_el.restore();
        });

        arCamera = new Camera(vid_el, {
            onFrame: async () => {
                if (window.cameraManager.currentFilter === 'ar') {
                    await mesh.send({image: vid_el});
                }
            },
            width: 640, 
            height: 480
        });
        
        arCamera.start();
        log_el.innerText = "¡Todo listo! Selecciona un filtro";
        
    } catch (err) {
        log_el.innerText = "ERROR de IA: " + err.message;
        console.error(err);
    }
}
</script>

---
### 18. ¿MediaPipe utiliza Sobel?
[cite_start]**No directamente.** MediaPipe utiliza Redes Neuronales Convolucionales (CNN)[cite: 158]. [cite_start]A diferencia del algoritmo Sobel, que utiliza fórmulas matemáticas fijas (kernels predefinidos) para buscar bordes, MediaPipe utiliza modelos entrenados con millones de imágenes para aprender a identificar patrones complejos como ojos, labios y contornos faciales, independientemente de los bordes simples[cite: 158].

# Análisis Matemático: Integración Unificada con MediaPipe Holistic

Para la etapa final del proyecto, nuestro equipo implementó la solución **MediaPipe Holistic**. El desafío matemático aquí no es solo detectar puntos, sino resolver el problema de la **Inferencia Jerárquica** para procesar 543 puntos de referencia en tiempo real (30 FPS) sin colapsar el procesador.

A continuación, describimos la arquitectura lógica que hemos desplegado:

### 1. Arquitectura de Tubería (Pipeline Jerárquico)
En lugar de ejecutar tres redes neuronales independientes (lo cual sería computacionalmente costoso), utilizamos un enfoque de **Regiones de Interés (ROI)** derivado de la geometría proyectiva.

1.  **Inferencia de Pose (Raíz):** Primero, el algoritmo detecta los 33 puntos del cuerpo.
2.  **Cálculo de ROI:** Basándose en la ubicación matemática de las muñecas y el cuello, el sistema calcula recortes (crops) rotados y escalados de la imagen original.
3.  **Inferencia Específica:** Estos recortes se alimentan a las sub-redes de **Manos** y **Face Mesh**.
    * *Ventaja Numérica:* Si las manos no son visibles en la etapa de Pose, el sistema ahorra recursos al no ejecutar la red de manos (Gating).

### 2. Espacios Vectoriales y Coordenadas 3D
El modelo nos devuelve vectores en un espacio métrico tridimensional $(x, y, z)$ para cada uno de los **543 landmarks**:
* **Pose:** 33 puntos.
* **Manos:** $21 \times 2 = 42$ puntos.
* **Rostro:** 468 puntos.

Para cada punto $P_i$, el modelo predice:
$$
P_i = [x, y, z, v]
$$
Donde:
* $x, y$: Coordenadas normalizadas $[0, 1]$ mapeadas al ancho y alto del canvas.
* $z$: Profundidad relativa. En el modelo de pose, el origen $(z=0)$ es el punto medio de las caderas. Valores negativos indican que el punto está más cerca de la cámara.
* $v$ (Visibilidad): Una probabilidad logística $[0, 1]$ que indica la certeza de que el punto es visible en la imagen (y no oculto por oclusión).

### 3. Topología de Malla Densa (Face Mesh)
Para el rostro, hemos activado `refineFaceLandmarks: true`. Esto utiliza una **Malla de Teselación** basada en la triangulación de Delaunay pre-calculada.
Matemáticamente, esto nos permite mapear la superficie curva del rostro en un plano 2D sin perder la coherencia topológica, permitiendo detectar micro-movimientos en labios y ojos con vectores de conexión específicos (`FACEMESH_LIPS`, `FACEMESH_RIGHT_EYE`, etc.).

### 4. Renderizado de Grafos Conectados
En la función de dibujo, tratamos los resultados como **Sub-Grafos Independientes**:
* **Árbol de Manos:** Un grafo acíclico donde la "muñeca" es el nodo raíz, ramificándose en 5 cadenas cinemáticas (dedos).
* **Malla Facial:** Un grafo cíclico denso diseñado para mantener la estructura estructural del óvalo facial.

Hemos implementado lógica condicional (`if (visibleParts.body)`, etc.) para optimizar el ciclo de renderizado, dibujando solo los tensores solicitados por el usuario y aplicando matrices de transformación de color distintas para diferenciar visualmente cada topología (Cian para cuerpo, Dorado para manos, Verde para rostro).




In [11]:
%%html
<div style="text-align: center; background: #1a1a1a; padding: 20px; border-radius: 15px; color: white; font-family: sans-serif;">
    <h3>MediaPipe</h3>
    <video id="v_pose_pro" width="640" height="480" style="display:none" playsinline></video>
    <canvas id="c_pose_pro" width="640" height="480" style="background: #000; border: 2px solid #9b59b6; border-radius: 10px;"></canvas>
    
    <div style="margin-top: 15px;">
        <button id="btn_on_pose" onclick="iniciarPosePro()" style="padding: 10px 20px; background: #9b59b6; color: white; border: none; border-radius: 5px; cursor: pointer; font-weight: bold;">ACTIVAR DETECCIÓN</button>
        <button id="btn_off_pose" onclick="detenerTodo()" style="padding: 10px 20px; background: #e74c3c; color: white; border: none; border-radius: 5px; cursor: pointer; margin-left: 10px; display: none;">APAGAR</button>
    </div>
    
    <div style="margin-top: 10px; display: flex; justify-content: center; gap: 5px; flex-wrap: wrap;">
        <button onclick="togglePart('body')" style="padding: 8px 15px; background: #3498db; color: white; border: none; border-radius: 5px; cursor: pointer;">Cuerpo</button>
        <button onclick="togglePart('hands')" style="padding: 8px 15px; background: #f39c12; color: white; border: none; border-radius: 5px; cursor: pointer;">Manos (21 puntos)</button>
        <button onclick="togglePart('face')" style="padding: 8px 15px; background: #e74c3c; color: white; border: none; border-radius: 5px; cursor: pointer;">Rostro (468 puntos)</button>
    </div>
    
    <p id="log_pose_pro" style="color: #f1c40f; font-size: 13px; margin-top: 10px;">Estado: Esperando cámara...</p>
    <p style="color: #95a5a6; font-size: 11px; margin-top: 5px;">Tip: Acerca tu mano o rostro para ver detalle máximo</p>
</div>

<script src="https://cdn.jsdelivr.net/npm/@mediapipe/camera_utils/camera_utils.js" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/drawing_utils/drawing_utils.js" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/holistic/holistic.js" crossorigin="anonymous"></script>

<script>
// Variables de configuración
let visibleParts = {
    body: true,
    hands: true,
    face: true
};

let holisticCamera = null;

function togglePart(part) {
    visibleParts[part] = !visibleParts[part];
}

async function iniciarPosePro() {
    const log = document.getElementById('log_pose_pro');
    
    detenerTodo();
    
    ['c_canny_pro', 'c_out', 'c_sobel_pro', 'c_pose_pro'].forEach(canvasId => {
        const canvas = document.getElementById(canvasId);
        if (canvas) {
            const ctx = canvas.getContext('2d');
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            ctx.fillStyle = '#000';
            ctx.fillRect(0, 0, canvas.width, canvas.height);
        }
    });
    
    try {
        log.innerText = "Solicitando cámara...";
        
        const stream = await navigator.mediaDevices.getUserMedia({ 
            video: { width: 640, height: 480, facingMode: 'user' } 
        });
        
        window.cameraManager.currentStream = stream;
        window.cameraManager.currentFilter = 'pose';
        
        const v = document.getElementById('v_pose_pro');
        const canvas = document.getElementById('c_pose_pro');
        const ctx = canvas.getContext('2d');
        
        v.srcObject = stream;
        await v.play();
        
        document.getElementById('btn_on_pose').style.display = 'none';
        document.getElementById('btn_off_pose').style.display = 'inline-block';
        
        log.innerText = "Cargando MediaPipe Holistic (cuerpo completo + manos + rostro)...";
        
        // Usar HOLISTIC para obtener TODO con alta precisión
        const holistic = new Holistic({
            locateFile: (file) => {
                return `https://cdn.jsdelivr.net/npm/@mediapipe/holistic/${file}`;
            }
        });
        
        holistic.setOptions({
            modelComplexity: 2,
            smoothLandmarks: true,
            enableSegmentation: false,
            smoothSegmentation: false,
            refineFaceLandmarks: true, // ¡CLAVE! Rostro detallado
            minDetectionConfidence: 0.5,
            minTrackingConfidence: 0.5
        });
        
        let frameCount = 0;
        
        holistic.onResults((results) => {
            if (window.cameraManager.currentFilter !== 'pose') return;
            
            ctx.save();
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            ctx.drawImage(results.image, 0, 0, canvas.width, canvas.height);
            
            let detections = [];
            
            // ========== CUERPO (33 puntos) ==========
            if (visibleParts.body && results.poseLandmarks) {
                // Esqueleto del cuerpo
                drawConnectors(ctx, results.poseLandmarks, POSE_CONNECTIONS, {
                    color: '#00FFFF',
                    lineWidth: 4
                });
                
                // Puntos del cuerpo
                drawLandmarks(ctx, results.poseLandmarks, {
                    color: '#00FFFF',
                    fillColor: '#00FFFF',
                    lineWidth: 2,
                    radius: 5
                });
                
                detections.push('Cuerpo (33 puntos)');
            }
            
            // ========== MANOS IZQUIERDA (21 puntos cada una) ==========
            if (visibleParts.hands) {
                // Mano izquierda
                if (results.leftHandLandmarks) {
                    drawConnectors(ctx, results.leftHandLandmarks, HAND_CONNECTIONS, {
                        color: '#FFD700',
                        lineWidth: 3
                    });
                    drawLandmarks(ctx, results.leftHandLandmarks, {
                        color: '#FFD700',
                        fillColor: '#FFFF00',
                        lineWidth: 2,
                        radius: 4
                    });
                    detections.push('Mano Izq (21 puntos)');
                }
                
                // Mano derecha
                if (results.rightHandLandmarks) {
                    drawConnectors(ctx, results.rightHandLandmarks, HAND_CONNECTIONS, {
                        color: '#FF8C00',
                        lineWidth: 3
                    });
                    drawLandmarks(ctx, results.rightHandLandmarks, {
                        color: '#FF8C00',
                        fillColor: '#FFA500',
                        lineWidth: 2,
                        radius: 4
                    });
                    detections.push('Mano Der (21 puntos)');
                }
            }
            
            // ========== ROSTRO (468 puntos - ¡SÍ, 468!) ==========
            if (visibleParts.face && results.faceLandmarks) {
                // Malla facial completa
                drawConnectors(ctx, results.faceLandmarks, FACEMESH_TESSELATION, {
                    color: '#00FF00',
                    lineWidth: 0.5
                });
                
                // Contornos importantes del rostro
                drawConnectors(ctx, results.faceLandmarks, FACEMESH_RIGHT_EYE, {
                    color: '#00FF00',
                    lineWidth: 2
                });
                drawConnectors(ctx, results.faceLandmarks, FACEMESH_LEFT_EYE, {
                    color: '#00FF00',
                    lineWidth: 2
                });
                drawConnectors(ctx, results.faceLandmarks, FACEMESH_LIPS, {
                    color: '#FF0000',
                    lineWidth: 2
                });
                drawConnectors(ctx, results.faceLandmarks, FACEMESH_FACE_OVAL, {
                    color: '#E0E0E0',
                    lineWidth: 2
                });
                
                // Puntos clave del rostro
                drawLandmarks(ctx, results.faceLandmarks, {
                    color: '#FF1493',
                    fillColor: '#FF69B4',
                    lineWidth: 1,
                    radius: 1
                });
                
                detections.push('Rostro (468 puntos)');
            }
            
            frameCount++;
            if (frameCount % 30 === 0) {
                const detectedText = detections.length > 0 ? detections.join(' + ') : 'Esperando...';
                log.innerText = `✅ ${detectedText} | Frames: ${frameCount}`;
            }
            
            ctx.restore();
        });
        
        // Iniciar cámara
        holisticCamera = new Camera(v, {
            onFrame: async () => {
                if (window.cameraManager.currentFilter === 'pose') {
                    await holistic.send({image: v});
                }
            },
            width: 640,
            height: 480
        });
        
        await holisticCamera.start();
        log.innerText = "Detección completa activa - Muestra tus manos y rostro";
        
    } catch (e) {
        log.innerText = "Error: " + e.message;
        console.error(e);
    }
}
</script>

---

### 19. ¿Qué son las redes neuronales convolucionales (CNN)?
[cite_start]Son un tipo de arquitectura de Deep Learning diseñada específicamente para procesar datos con estructura de cuadrícula, como las imágenes[cite: 160]. [cite_start]

[Image of convolutional neural network architecture]
 Utilizan capas de "convolución" que funcionan como filtros aprendidos automáticamente[cite: 161]. [cite_start]A diferencia de Sobel (donde el humano define el filtro), una CNN aprende por sí sola qué filtros aplicar para detectar desde líneas simples hasta formas complejas como una cara humana[cite: 161].

### 23. Escribir el mismo concepto pero usando Sobel
[cite_start]Implementación de procesamiento de video en tiempo real utilizando el operador Sobel para detección de bordes mediante OpenCV.js[cite: 163].

# Análisis Matemático: Implementación Manual del Operador Sobel

En este módulo, nuestro equipo ha programado el algoritmo de Sobel manipulando directamente los buffers de memoria de la imagen. El objetivo es calcular el **Gradiente de Intensidad** de la imagen, que es un vector que apunta en la dirección del mayor cambio de brillo (el borde).

A continuación, describimos las operaciones algebraicas que realizamos paso a paso en el código:

### 1. Reducción Dimensional (Escala de Grises)
La primera etapa consiste en transformar el espacio de color tridimensional ($R, G, B$) en un campo escalar bidimensional $I(x,y)$. Utilizamos la combinación lineal estándar para la luminancia:

$$
I(x,y) = 0.299 \cdot R + 0.587 \cdot G + 0.114 \cdot B
$$

> **En nuestro código:** Iteramos por el array `data` leyendo de 4 en 4 (RGBA) y guardamos el resultado en el array `gray`.

---

### 2. Derivación Discreta (Máscaras de Convolución)
Para encontrar los bordes, necesitamos calcular la derivada de la intensidad. Como la imagen es discreta, utilizamos **Diferencias Finitas Centrales** ponderadas.

Implementamos dos núcleos (kernels) de convolución de $3 \times 3$. En el bucle `for` anidado, para cada píxel $(x,y)$, operamos sobre su **Vecindad de 8-conexión**:

#### A. Gradiente Horizontal ($G_x$)
Calcula la diferencia de intensidad de izquierda a derecha. Nuestro código aplica explícitamente la siguiente matriz:

$$
G_x = \begin{bmatrix} 
-1 & 0 & +1 \\ 
-2 & 0 & +2 \\ 
-1 & 0 & +1 
\end{bmatrix} * I
$$

> **Lógica del código:** `(gray[derecha] - gray[izquierda])`. Los píxeles centrales tienen peso 2 para suavizar el ruido local.

#### B. Gradiente Vertical ($G_y$)
Calcula la diferencia de intensidad de arriba a abajo.

$$
G_y = \begin{bmatrix} 
-1 & -2 & -1 \\ 
0 & 0 & 0 \\ 
+1 & +2 & +1 
\end{bmatrix} * I
$$

> **Lógica del código:** `(gray[abajo] - gray[arriba])`.

---

### 3. Cálculo de la Magnitud (Norma Euclidiana)
Una vez obtenidos los componentes vectoriales $G_x$ y $G_y$, necesitamos la intensidad total del borde.
En esta implementación, hemos optado por la precisión matemática utilizando la **Norma Euclidiana ($L_2$)**:

$$
|G| = \sqrt{G_x^2 + G_y^2}
$$

> **En nuestro código:** La línea `const mag = Math.sqrt(gx * gx + gy * gy)` realiza este cálculo pitagórico. Finalmente, limitamos el valor a 255 (`Math.min`) para visualizarlo correctamente en el monitor.

### Conclusión del Algoritmo
El resultado visual es un **Mapa de Magnitudes**: las zonas planas (color negro) tienen derivada cercana a cero, mientras que los cambios bruscos (bordes) tienen derivadas altas, apareciendo en blanco.

In [19]:
%%html
<div style="text-align: center; background: #1a1a1a; padding: 20px; border-radius: 15px; color: white; font-family: sans-serif;">
    <h3>Método Sobel (Gradientes de Intensidad)</h3>
    <video id="v_sobel_pro" width="640" height="480" style="display:none" playsinline></video>
    <canvas id="c_sobel_pro" width="640" height="480" style="background: #000; border: 2px solid #2980b9; border-radius: 10px;"></canvas>
    <div style="margin-top: 15px;">
        <button id="btn_on_sobel" onclick="iniciarSobelPro()" style="padding: 10px 20px; background: #2980b9; color: white; border: none; border-radius: 5px; cursor: pointer; font-weight: bold;">ACTIVAR SOBEL</button>
        <button id="btn_off_sobel" onclick="detenerTodo()" style="padding: 10px 20px; background: #e74c3c; color: white; border: none; border-radius: 5px; cursor: pointer; margin-left: 10px; display: none;">APAGAR</button>
    </div>
    <p id="log_sobel_pro" style="color: #f1c40f; font-size: 13px; margin-top: 10px;">Estado: Esperando cámara...</p>
</div>

<script>
// Implementación de Sobel
function applySobelGradients(imageData) {
    const width = imageData.width;
    const height = imageData.height;
    const data = imageData.data;
    
    // Convertir a escala de grises
    const gray = new Uint8ClampedArray(width * height);
    for (let i = 0; i < data.length; i += 4) {
        const idx = i / 4;
        gray[idx] = 0.299 * data[i] + 0.587 * data[i + 1] + 0.114 * data[i + 2];
    }
    
    // Aplicar Sobel
    const gradX = new Float32Array(width * height);
    const gradY = new Float32Array(width * height);
    const magnitude = new Uint8ClampedArray(width * height);
    
    for (let y = 1; y < height - 1; y++) {
        for (let x = 1; x < width - 1; x++) {
            const idx = y * width + x;
            
            // Sobel X (detecta bordes verticales)
            const gx = (
                -gray[(y-1)*width + (x-1)] + gray[(y-1)*width + (x+1)] +
                -2*gray[y*width + (x-1)] + 2*gray[y*width + (x+1)] +
                -gray[(y+1)*width + (x-1)] + gray[(y+1)*width + (x+1)]
            );
            
            // Sobel Y (detecta bordes horizontales)
            const gy = (
                -gray[(y-1)*width + (x-1)] - 2*gray[(y-1)*width + x] - gray[(y-1)*width + (x+1)] +
                gray[(y+1)*width + (x-1)] + 2*gray[(y+1)*width + x] + gray[(y+1)*width + (x+1)]
            );
            
            gradX[idx] = gx;
            gradY[idx] = gy;
            
            // Magnitud del gradiente
            const mag = Math.sqrt(gx * gx + gy * gy);
            magnitude[idx] = Math.min(255, mag);
        }
    }
    
    // Convertir a ImageData
    const output = new ImageData(width, height);
    for (let i = 0; i < magnitude.length; i++) {
        output.data[i * 4] = magnitude[i];
        output.data[i * 4 + 1] = magnitude[i];
        output.data[i * 4 + 2] = magnitude[i];
        output.data[i * 4 + 3] = 255;
    }
    
    return output;
}

async function iniciarSobelPro() {
    const log = document.getElementById('log_sobel_pro');
    
    // Detener otros filtros Y limpiar sus canvas
    detenerTodo();
    
    // IMPORTANTE: Limpiar TODOS los canvas antes de empezar
    ['c_canny_pro', 'c_out', 'c_sobel_pro'].forEach(canvasId => {
        const canvas = document.getElementById(canvasId);
        if (canvas) {
            const ctx = canvas.getContext('2d');
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            ctx.fillStyle = '#000';
            ctx.fillRect(0, 0, canvas.width, canvas.height);
        }
    });
    
    try {
        log.innerText = "📷 Solicitando cámara...";
        
        const stream = await navigator.mediaDevices.getUserMedia({ 
            video: { width: 640, height: 480, facingMode: 'user' } 
        });
        
        window.cameraManager.currentStream = stream;
        window.cameraManager.currentFilter = 'sobel';
        
        const v = document.getElementById('v_sobel_pro');
        const canvas = document.getElementById('c_sobel_pro');
        const ctx = canvas.getContext('2d');
        
        v.srcObject = stream;
        await v.play();
        
        await new Promise(resolve => setTimeout(resolve, 500));
        
        document.getElementById('btn_on_sobel').style.display = 'none';
        document.getElementById('btn_off_sobel').style.display = 'inline-block';
        log.innerText = "Gradientes Sobel activos";
        
        window.cameraManager.activeLoop = true;
        let frameCount = 0;
        
        function processFrame() {
            if (!window.cameraManager.activeLoop || window.cameraManager.currentFilter !== 'sobel') {
                return;
            }
            
            try {
                ctx.drawImage(v, 0, 0, canvas.width, canvas.height);
                const imageData = ctx.getImageData(0, 0, canvas.width, canvas.height);
                
                // Aplicar SOBEL
                const edges = applySobelGradients(imageData);
                
                ctx.putImageData(edges, 0, 0);
                
                frameCount++;
                if (frameCount % 30 === 0) {
                    log.innerText = "Sobel activo - Frames: " + frameCount;
                }
                
                requestAnimationFrame(processFrame);
                
            } catch (e) {
                console.error('Error:', e);
                log.innerText = "Error: " + e.message;
            }
        }
        
        processFrame();
        
    } catch (e) {
        log.innerText = "Error: " + e.message;
        console.error(e);
    }
}
</script>

### Codigo Three.js

# Introducción a Three.js + Face Mesh (AR)

Este proyecto es un ejemplo práctico de **Realidad Aumentada (AR)** en la web. Utiliza la potencia de los gráficos 3D junto con inteligencia artificial para crear una experiencia interactiva en tiempo real.

---

## 1. El Matrimonio de dos Tecnologías
El código funciona gracias a la colaboración de dos herramientas poderosas:

* **Three.js:** Se encarga de la **capa visual**. Crea la esfera amarilla, gestiona las luces, la cámara y renderiza los gráficos a alta velocidad directamente en el navegador.
* **MediaPipe (Face Mesh):** Es el **cerebro de IA** desarrollado por Google. Analiza el flujo de video de tu cámara para identificar 468 puntos clave de tu rostro en milisegundos.



---

## 2. El Flujo Lógico del Proceso
Para que la esfera "persiga" tu nariz, el código ejecuta un ciclo continuo de cuatro pasos:

1.  **Captura:** Se solicita acceso a la webcam y se genera un elemento de `<video>` que sirve como fuente de datos.
2.  **Detección (IA):** MediaPipe recibe cada fotograma del video, localiza el punto específico de tu nariz y genera coordenadas matemáticas $x, y, z$.
3.  **Traducción:** El script convierte esas coordenadas del mundo real (píxeles de la cámara) a las coordenadas del mundo 3D de la escena de Three.js.
4.  **Renderizado:** Three.js redibuja la esfera en la nueva posición aproximadamente **60 veces por segundo** (60 FPS), creando una sensación de movimiento fluido y natural.

---

## 3. Anatomía del Objeto 3D
Dentro de la función `crearEsferaGuia()`, definimos el "ADN" del objeto usando dos componentes base:

| Componente | Definición en Código | Descripción |
| :--- | :--- | :--- |
| **Geometría** | `IcosahedronGeometry(30, 2)` | Define la estructura física. Es un icosaedro con subdivisiones que le dan forma de esfera facetada. |
| **Material** | `MeshBasicMaterial` | Define la apariencia. El color `0xffff00` (amarillo) y la propiedad `wireframe: true` le dan ese estilo de "malla" o holograma. |



---

## 4. ¿Por qué es especial este código?
A diferencia de una animación 3D tradicional, este sistema es **contextual e interactivo**:

* **Capas Superpuestas:** Utiliza un `<canvas>` transparente colocado exactamente encima del video de la cámara.
* **Cámara Ortográfica:** Se usa `OrthographicCamera` para evitar que el objeto se deforme por la perspectiva, facilitando que la esfera encaje perfectamente con tu rostro en una pantalla 2D.
* **Eficiencia:** Todo el procesamiento ocurre en el dispositivo del usuario (Client-side), lo que garantiza privacidad y rapidez sin depender de un servidor externo.

---
> **Nota técnica:** Este flujo es la base para crear filtros de Instagram o aplicaciones de probadores virtuales (gafas, sombreros, maquillaje) directamente en la web.

# Análisis Matemático: Integración de Gráficos 3D con Three.js

En esta sección, nuestro objetivo era renderizar objetos tridimensionales interactivos que respondieran a la geometría del rostro en tiempo real. Para ello, utilizamos la librería **Three.js**, que nos permite manipular vértices y matrices en un entorno WebGL.

El desafío principal fue la **Sincronización de Coordenadas**: MediaPipe entrega coordenadas normalizadas (2D+Z relativo), mientras que Three.js trabaja en un espacio métrico euclidiano (3D absoluto).

A continuación, detallamos la lógica matemática de la integración:

### 1. Configuración de la Escena (Proyección Ortográfica)
Dado que estamos trabajando sobre un video 2D plano, utilizamos una **Cámara Ortográfica** en lugar de una de Perspectiva.
Matemáticamente, esto elimina la deformación por punto de fuga, garantizando que el tamaño de los objetos dependa exclusivamente de nuestra lógica de escala y no de la distancia virtual de la cámara 3D.

$$
P_{proyectado} = \begin{bmatrix} 
\frac{2}{r-l} & 0 & 0 & -\frac{r+l}{r-l} \\
0 & \frac{2}{t-b} & 0 & -\frac{t+b}{t-b} \\
0 & 0 & -\frac{2}{f-n} & -\frac{f+n}{f-n} \\
0 & 0 & 0 & 1 
\end{bmatrix} \cdot P_{mundo}
$$

> **En nuestro código:** Configuramos los límites `left=-320`, `right=320`, `top=240`, `bottom=-240` para que coincidan píxel a píxel con la resolución del video ($640 \times 480$).

### 2. Mapeo de Coordenadas (Transformación Afín)
El núcleo del algoritmo es la función que traduce la posición de la nariz (Landmark #4) al espacio 3D de Three.js.

1.  **Centrado del Eje:** MediaPipe tiene su origen $(0,0)$ en la esquina superior izquierda. Three.js tiene su origen $(0,0)$ en el centro exacto de la pantalla.
    * **Ecuación de traslación X:** $x_{3D} = (x_{MP} - 0.5) \cdot Ancho$
    * **Ecuación de traslación Y:** $y_{3D} = -(y_{MP} - 0.5) \cdot Alto$ (Invertimos el signo porque en 3D el eje Y positivo suele ir hacia arriba).

2.  **Profundidad (Eje Z):** MediaPipe entrega un valor Z relativo a la escala de la cara. Lo multiplicamos por un factor escalar empírico ($-200$) para que la esfera parezca acercarse o alejarse correctamente.

### 3. Renderizado de la Esfera (Geometría Icosaédrica)
Para la "Esfera Guía", utilizamos un **Icosaedro** (`IcosahedronGeometry`) subdividido 2 veces.
* **Topología:** Es un poliedro convexo regular. Al activar `wireframe: true`, el motor gráfico solo renderiza las aristas del grafo poliedral, permitiendo ver a través del objeto y apreciar su rotación.
* **Animación:** En cada frame de renderizado, aplicamos una matriz de rotación incremental:
    $$
    R_{total} = R_y(\theta + 0.02) \cdot R_x(\phi + 0.01)
    $$
    Esto genera un giro constante que ayuda a visualizar la tridimensionalidad del objeto sobre el video plano.

In [5]:
%%html
<div style="text-align: center; background: #1a1a1a; padding: 20px; border-radius: 15px; color: white; font-family: sans-serif;">
    <h3>Three AR (Esfera Guía)</h3>
    <p style="font-size: 12px; color: #aaa;">La esfera te esta siguiendo</p>
    
    <div id="threejs_container_sphere" style="position: relative; display: inline-block; width: 640px; height: 480px; border: 2px solid #555; border-radius: 10px; overflow: hidden; background: #000;">
        </div>
    
    <div style="margin-top: 15px;">
        <button id="btn_on_sphere" onclick="iniciarSphereAR()" style="padding: 10px 20px; background: #f1c40f; color: #000; border: none; border-radius: 5px; cursor: pointer; font-weight: bold;">ACTIVAR ESFERA</button>
        <button id="btn_off_sphere" onclick="detenerSphereAR()" style="padding: 10px 20px; background: #e74c3c; color: white; border: none; border-radius: 5px; cursor: pointer; margin-left: 10px; display: none;">APAGAR</button>
    </div>

    
    <div style="margin-top: 10px;">
        <button id="toggle_bg_sphere" onclick="toggleBackgroundSphere()" style="padding: 8px 15px; background: #34495e; color: white; border: none; border-radius: 5px; cursor: pointer; font-size: 12px;">Activar camara</button>
    </div>
    
    <p id="log_sphere" style="color: #f1c40f; font-size: 13px; margin-top: 10px;">Estado: Esperando cámara...</p>
</div>

<script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/camera_utils/camera_utils.js" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/face_mesh.js" crossorigin="anonymous"></script>

<script>
let tScene, tCamera, tRenderer, tObject;
let tCameraInstance = null;
let tVideoElement = null;
let isVideoVisible = false; // Estado inicial: Video oculto

// --- FUNCIÓN CORREGIDA PARA MOSTRAR/OCULTAR VIDEO ---
function toggleBackgroundSphere() {
    // 1. Buscamos el elemento de video directamente por su ID para asegurar que existe
    const vid = document.getElementById('v_threejs_sphere');
    const btn = document.getElementById('toggle_bg_sphere');
    
    if (vid) {
        isVideoVisible = !isVideoVisible;
        // Cambiamos el estilo display entre 'block' (visible) y 'none' (oculto)
        vid.style.display = isVideoVisible ? 'block' : 'none';
        // Actualizamos el texto del botón
        if (btn) btn.innerText = isVideoVisible ? 'Ocultar Mi Cara' : 'Mostrar Mi Cara';
    } else {
        console.log("El video aún no está listo. Inicia la cámara primero.");
    }
}

// Función para crear la esfera amarilla de alambre (igual que tu imagen)
function crearEsferaGuia() {
    const geometry = new THREE.IcosahedronGeometry(30, 2); // Esfera geodésica
    const material = new THREE.MeshBasicMaterial({ 
        color: 0xffff00,       // Amarillo
        wireframe: true        // Modo alambre
    });
    return new THREE.Mesh(geometry, material);
}

// Limpieza de memoria
function detenerSphereAR() {
    if (tCameraInstance) {
        try { tCameraInstance.stop(); } catch(e) { console.log(e); }
        tCameraInstance = null;
    }
    
    if (tRenderer) { tRenderer.dispose(); tRenderer = null; }
    if (tScene) {
        while(tScene.children.length > 0) { tScene.remove(tScene.children[0]); }
        tScene = null;
    }
    
    if (window.cameraManager && window.cameraManager.currentStream) {
        window.cameraManager.currentStream.getTracks().forEach(t => t.stop());
        window.cameraManager.currentStream = null;
    }
    
    if (tVideoElement) {
        if (tVideoElement.srcObject) {
            tVideoElement.srcObject.getTracks().forEach(t => t.stop());
            tVideoElement.srcObject = null;
        }
        tVideoElement = null;
    }
    
    // Reseteamos estado del botón
    isVideoVisible = false;
    const btn = document.getElementById('toggle_bg_sphere');
    if(btn) btn.innerText = 'Mostrar Mi Cara';
    
    const container = document.getElementById('threejs_container_sphere');
    if (container) container.innerHTML = '';
    
    document.getElementById('btn_on_sphere').style.display = 'inline-block';
    document.getElementById('btn_off_sphere').style.display = 'none';
    document.getElementById('log_sphere').innerText = "Sistema apagado.";
}

async function iniciarSphereAR() {
    const log = document.getElementById('log_sphere');
    
    try {
        log.innerText = "Iniciando sistema...";
        detenerSphereAR(); // Limpieza previa
        await new Promise(resolve => setTimeout(resolve, 300));
        
        const container = document.getElementById('threejs_container_sphere');
        
        // 1. CREAMOS EL VIDEO (Oculto por defecto con display: none)
        tVideoElement = document.createElement('video');
        tVideoElement.id = 'v_threejs_sphere'; // ID importante para el botón
        tVideoElement.width = 640;
        tVideoElement.height = 480;
        tVideoElement.autoplay = true;
        tVideoElement.playsInline = true;
        tVideoElement.muted = true;
        // style.display = none inicial
        tVideoElement.style.cssText = 'position: absolute; top: 0; left: 0; transform: scaleX(-1); width: 100%; height: 100%; display: none;';
        container.appendChild(tVideoElement);
        
        // 2. CREAMOS EL CANVAS (Donde se dibuja la esfera)
        const tCanvas = document.createElement('canvas');
        tCanvas.id = 'c_threejs_sphere';
        tCanvas.width = 640;
        tCanvas.height = 480;
        // z-index alto para estar encima
        tCanvas.style.cssText = 'position: absolute; top: 0; left: 0; transform: scaleX(-1); width: 100%; height: 100%; z-index: 10;';
        container.appendChild(tCanvas);
        
        log.innerText = "Accediendo a cámara...";
        
        const stream = await navigator.mediaDevices.getUserMedia({ 
            video: { width: 640, height: 480, facingMode: 'user' } 
        });
        
        if (!window.cameraManager) window.cameraManager = {};
        window.cameraManager.currentStream = stream;
        window.cameraManager.currentFilter = 'sphere_ar';
        
        tVideoElement.srcObject = stream;
        await tVideoElement.play();
        
        await new Promise(resolve => {
            if (tVideoElement.videoWidth > 0) resolve();
            else tVideoElement.onloadedmetadata = () => resolve();
        });
        
        document.getElementById('btn_on_sphere').style.display = 'none';
        document.getElementById('btn_off_sphere').style.display = 'inline-block';
        
        log.innerText = "Configurando 3D...";
        
        // ========== CONFIGURAR THREE.JS ==========
        tScene = new THREE.Scene();
        tCamera = new THREE.OrthographicCamera(-320, 320, 240, -240, 0.1, 1000);
        tCamera.position.z = 10;
        
        tRenderer = new THREE.WebGLRenderer({ 
            canvas: tCanvas, 
            alpha: true, // Fondo transparente
            antialias: true 
        });
        tRenderer.setSize(640, 480);
        
        // Agregar la Esfera Amarilla
        tObject = crearEsferaGuia();
        tObject.visible = false;
        tScene.add(tObject);
        
        log.innerText = "Activando seguimiento...";
        
        // ========== CONFIGURAR MEDIAPIPE ==========
        const faceMesh = new FaceMesh({
            locateFile: (file) => `https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/${file}`
        });
        
        faceMesh.setOptions({
            maxNumFaces: 1,
            refineLandmarks: true,
            minDetectionConfidence: 0.5,
            minTrackingConfidence: 0.5
        });
        
        faceMesh.onResults((results) => {
            if (window.cameraManager.currentFilter !== 'sphere_ar' || !tRenderer) return;
            
            tRenderer.clear();
            
            if (results.multiFaceLandmarks && results.multiFaceLandmarks.length > 0) {
                const landmarks = results.multiFaceLandmarks[0];
                const nose = landmarks[4]; // Punta de la nariz
                
                const x = (nose.x - 0.5) * 640;
                const y = -(nose.y - 0.5) * 480;
                const z = -nose.z * 200;
                
                tObject.position.set(x, y, z);
                tObject.rotation.x += 0.01;
                tObject.rotation.y += 0.02;
                
                tObject.visible = true;
                log.innerText = "Esfera activa siguiendo nariz";
            } else {
                tObject.visible = false;
                log.innerText = "Rostro no detectado";
            }
            
            tRenderer.render(tScene, tCamera);
        });
        
        // ========== INICIAR CÁMARA ==========
        tCameraInstance = new Camera(tVideoElement, {
            onFrame: async () => {
                if (window.cameraManager.currentFilter === 'sphere_ar') {
                    await faceMesh.send({image: tVideoElement});
                }
            },
            width: 640,
            height: 480
        });
        
        await tCameraInstance.start();
        
    } catch (e) {
        log.innerText = "Error: " + e.message;
        console.error(e);
        detenerSphereAR();
    }
}
</script>